<a href="https://colab.research.google.com/github/marcos-silva-inf/DeepClusteringSpatialPanelData/blob/main/python/00_transform_raw_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import h5py
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
import keras.backend as K
import csv
from keras import layers
from sklearn.model_selection import train_test_split
from keras import regularizers


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/DATA/PAPER2023/data/raw-data-entropy-diversity-indices_v2.csv', sep=';', low_memory=False)

var = [0,1]
herd = list(range(12, 23))
var.extend(herd)
prodani = list(range(23, 28))
var.extend(prodani)
vl_t = list(range(96, 127))
var.extend(vl_t)
vl_p = list(range(127, 163))
var.extend(vl_p)
plant_t = list(range(163, 194))
var.extend(plant_t)
aqua = list(range(230, 254))
var.extend(aqua)
ext = list(range(254, 297))
var.extend(ext)
silv = list(range(297, 312))
var.extend(silv)

df = df.fillna(0)

def add_T(row, v):
   tmp_total = 0
   for i in range(len(v)):
      tmp_total = tmp_total + row[v[i]]
   return tmp_total

df['T1'] = df.apply(add_T, v=herd, axis=1)
df['T2'] = df.apply(add_T, v=prodani, axis=1)
df['T3'] = df.apply(add_T, v=vl_t, axis=1)
df['T4'] = df.apply(add_T, v=vl_p, axis=1)
df['T5'] = df.apply(add_T, v=plant_t, axis=1)
df['T6'] = df.apply(add_T, v=aqua, axis=1)
df['T7'] = df.apply(add_T, v=ext, axis=1)
df['T8'] = df.apply(add_T, v=silv, axis=1)

df.iloc[:, herd] = df.iloc[:,herd].div(df.T1, axis=0).replace(np.inf, 0)
df.iloc[:, prodani] = df.iloc[:,prodani].div(df.T2, axis=0).replace(np.inf, 0)
df.iloc[:, vl_t] = df.iloc[:,vl_t].div(df.T3, axis=0).replace(np.inf, 0)
df.iloc[:, vl_p] = df.iloc[:,vl_p].div(df.T4, axis=0).replace(np.inf, 0)
df.iloc[:, plant_t] = df.iloc[:,plant_t].div(df.T5, axis=0).replace(np.inf, 0)
df.iloc[:, aqua] = df.iloc[:,aqua].div(df.T6, axis=0).replace(np.inf, 0)
df.iloc[:, ext] = df.iloc[:,ext].div(df.T7, axis=0).replace(np.inf, 0)
df.iloc[:, silv] = df.iloc[:,silv].div(df.T8, axis=0).replace(np.inf, 0)

df = df.fillna(0)

panel_data = df.iloc[:,var]

var_list = list(range(2, 198))
panel_data_num = panel_data.iloc[:,var_list]


normalized_panel_data=(panel_data_num-panel_data_num.min())/(panel_data_num.max()-panel_data_num.min())

result = pd.concat([panel_data.iloc[:,[0,1]], normalized_panel_data.iloc[:,:]], axis=1, join="inner")

id = result.CODIBGE.unique()
colnames = result.columns

data_image = np.zeros((5570,20,196))

m = result.shape[0]
for i in range(m):
  if (i + 1) % 20 == 0:
    j = 19
  else:
    j = i + 1 - 20*((i + 1) // 20) - 1
  index = (i + 1) // 20 - 1
  for k in range(196) :
      data_image[index][j][k] = result.loc[ i , colnames[k + 2]]  

id = id.reshape(5570,1)

In [ ]:
result.to_csv("/content/drive/MyDrive/DATA/PAPER2023/data/raw-data-min-max-normalized.csv")